# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import warnings 
warnings.filterwarnings("ignore")

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,53.82,98,100,4.00,NZ,1690226994
1,1,adamstown,-25.0660,-130.1015,67.93,68,29,9.42,PN,1690226995
2,2,saint-joseph,-21.3667,55.6167,66.58,83,39,7.05,RE,1690226693
3,3,carnarvon,-24.8667,113.6333,60.48,76,4,10.89,AU,1690226996
4,4,hermanus,-34.4187,19.2345,51.93,66,0,5.99,ZA,1690226997


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
humidity_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    size = "Humidity",
    geo = True,
    tiles = "OSM",
    color = "Humidity") #open street map

# Display the map
humidity_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

In [4]:
# !pip freeze #takes inventory of what I have installed

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather = city_data_df.loc[(city_data_df["Max Temp"] <= 70)&(city_data_df["Cloudiness"] <= 25)
                                 &(city_data_df["Max Temp"] >= 50)]

#sign for "or" is |  and & (and is for we want both to be true)

# Drop any rows with null values
#null values is if missing some kind of value 
ideal_weather = ideal_weather.dropna(how="any") 
#how = all means all the rows have to be empty in order to be dropped
#how = any means if any one of the info is missing it will be dropped

# Display sample data
ideal_weather.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,carnarvon,-24.8667,113.6333,60.48,76,4,10.89,AU,1690226996
4,4,hermanus,-34.4187,19.2345,51.93,66,0,5.99,ZA,1690226997
8,8,klyuchi,52.2667,79.1667,68.88,57,7,8.84,RU,1690226999
11,11,margaret river,-33.9500,115.0667,61.84,80,11,20.04,AU,1690227001
16,16,saint-pierre,-21.3393,55.4781,67.68,77,0,6.91,RE,1690227006


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather[["City", "Country", "Lat", "Lng", "Humidity"]]
hotel_df

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
3,carnarvon,AU,-24.8667,113.6333,76,
4,hermanus,ZA,-34.4187,19.2345,66,
8,klyuchi,RU,52.2667,79.1667,57,
11,margaret river,AU,-33.9500,115.0667,80,
16,saint-pierre,RE,-21.3393,55.4781,77,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [10]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
   
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
       
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
carnarvon - nearest hotel: No hotel found
hermanus - nearest hotel: Aloe guest house
klyuchi - nearest hotel: No hotel found
margaret river - nearest hotel: Margaret River Hotel
saint-pierre - nearest hotel: Tropic Hotel
murghob - nearest hotel: Pamir Hotel
villa huidobro - nearest hotel: No hotel found
kalamunda - nearest hotel: No hotel found
ghanzi - nearest hotel: Ghanzi farmhouse
petropavlovsk-kamchatsky - nearest hotel: ООО Постоялый двор
kuanda - nearest hotel: No hotel found
aldinga beach - nearest hotel: The Beach Hotel
tocopilla - nearest hotel: hotel Isidora
pocito - nearest hotel: No hotel found
usinsk - nearest hotel: общежитие Лукойл-Коми
sitka - nearest hotel: Aspen Suites Hotel
ola - nearest hotel: No hotel found
olyokminsk - nearest hotel: Колос
nuuk - nearest hotel: Nordbo
nuku'alofa - nearest hotel: City Hotel
black point - nearest hotel: No hotel found
port alfred - nearest hotel: No hotel found
udachny - nearest hotel: Вилюй
northam - nearest 

,City,Country,Lat,Lng,Humidity,Hotel Name
3,carnarvon,AU,-24.8667,113.6333,76,No hotel found
4,hermanus,ZA,-34.4187,19.2345,66,Aloe guest house
8,klyuchi,RU,52.2667,79.1667,57,No hotel found
11,margaret river,AU,-33.9500,115.0667,80,Margaret River Hotel
16,saint-pierre,RE,-21.3393,55.4781,77,Tropic Hotel
22,murghob,TJ,38.1702,73.9667,30,Pamir Hotel
28,villa huidobro,AR,-34.8383,-64.5869,35,No hotel found
89,kalamunda,AU,-32.0040,116.1329,59,No hotel found
92,ghanzi,BW,-21.5667,21.7833,29,Ghanzi farmhouse
95,petropavlovsk-kamchatsky,RU,53.0452,158.6483,88,ООО Постоялый двор


In [13]:
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != "No hotel found"]
clean_hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
4,hermanus,ZA,-34.4187,19.2345,66,Aloe guest house
11,margaret river,AU,-33.9500,115.0667,80,Margaret River Hotel
16,saint-pierre,RE,-21.3393,55.4781,77,Tropic Hotel
22,murghob,TJ,38.1702,73.9667,30,Pamir Hotel
92,ghanzi,BW,-21.5667,21.7833,29,Ghanzi farmhouse
95,petropavlovsk-kamchatsky,RU,53.0452,158.6483,88,ООО Постоялый двор
105,aldinga beach,AU,-35.2793,138.4604,84,The Beach Hotel
119,tocopilla,CL,-22.0920,-70.1979,67,hotel Isidora
130,usinsk,RU,65.9939,57.5281,78,общежитие Лукойл-Коми
131,sitka,US,57.0531,-135.3300,77,Aspen Suites Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [17]:
%%capture --no-display

# Configure the map plot
hotel_plot = clean_hotel_df.hvplot.points(
    "Lng",
    "Lat",
    size = "Humidity",
    geo = True,
    tiles = "OSM",
    color = "Country",
    hover_cols = ["Hotel Name", "Country"])

# Display the map
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Country,Humidity,Hotel Name)